In [5]:
# START GENAI
import requests
import json
import datetime
import random

from collections import defaultdict
from collections import namedtuple

In [6]:
url = 'https://api.entur.io/journey-planner/v3/graphql'

headers = {
    'accept': 'application/json, text/plain, */*',
    'et-client-name': 'private-dashboard',
    'X-contact': 'cloessl@gmail.com'
}

current_time = datetime.datetime.now(datetime.UTC)
future_time = current_time + datetime.timedelta(minutes=3)
formatted_time = future_time.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'

start_time = formatted_time
fetch_limit = 200
minute_to_fetch = 30*60
entur_stop = "NSR:StopPlace:58189"

payload = {
    "query": f"""
    fragment estimatedCallsParts on EstimatedCall {{
        destinationDisplay {{
            frontText
        }}
        situations {{
            summary {{
                value
                language
            }}
        }}
        quay {{
            publicCode
        }}
        expectedDepartureTime
        actualDepartureTime
        aimedDepartureTime
        serviceJourney {{
            line {{
                publicCode
                transportMode
            }}
        }}
    }}
    query {{
        board1: stopPlaces(ids: ["{entur_stop}"]) {{
            name
            estimatedCalls(startTime: "{start_time}" whiteListedModes: [rail,bus,metro,tram,water,coach], numberOfDepartures: {fetch_limit}, arrivalDeparture: departures, includeCancelledTrips: true, timeRange: {minute_to_fetch}) {{
                ...estimatedCallsParts
            }}
        }}
    }}
    """
}

response = requests.post(url, headers=headers, json=payload)

jdata = response.json()
data = jdata['data']['board1'][0]['estimatedCalls']

print(f"Setting max res: {fetch_limit}")
print(f"Setting minutes: {int(minute_to_fetch/60)}")
print(f"Current time   : {current_time}")
print(f"Future    time : {future_time}")
print(f"Requested time : {formatted_time}")

print("Status Code:", response.status_code)

print(f"Res, dep. times: {len(jdata['data']['board1'][0]['estimatedCalls'])}")

Setting max res: 200
Setting minutes: 30
Current time   : 2025-02-18 09:18:43.182722+00:00
Future    time : 2025-02-18 09:21:43.182722+00:00
Requested time : 2025-02-18T09:21:43.182Z
Status Code: 200
Res, dep. times: 64


In [7]:
# import json

# pretty_json = json.dumps(jdata, indent=2)
# print(pretty_json)

In [8]:
def convert_bus_dep_times(iso_date_time: str) -> str:
    from datetime import datetime
    dt = datetime.fromisoformat(iso_date_time)
    return dt.strftime("%H:%M.%S")


def sort_key(item):
    bus_line = item[0].line
    platform = item[0].platform
    bus_dest = item[0].dst
    local_first = isinstance(bus_line, str)  # Make sure lined routes are before FB1, etc.
    return (local_first, bus_line, platform, bus_dest)
    

def rnd(dictionary):
    key = random.choice(list(dictionary.keys()))
    print(type(key))
    print(type(dictionary[key]))
    print(key)
    print(dictionary[key])


def print_dep_times(list):
    for (line, line_dst, platform), items in list:
        trans_type = items[0]['type']
        print(f"Line: {line}, To: {line_dst}, Platform: {platform}, Type: {trans_type}")
        for item in items:
            # print(f"  {item}")
            print(f"\t{item['schedule']} - {item['expected']}")

In [9]:
def create_stripped_item(data):
    stripped_item = {
        # "dest": data["destinationDisplay"]["frontText"],
        # "situations": data["situations"],
        # "plat": data["quay"]["publicCode"],
        "schedule": convert_bus_dep_times(data["aimedDepartureTime"]),
        "expected": convert_bus_dep_times(data["expectedDepartureTime"]),
        # "line": data["serviceJourney"]["line"]["publicCode"],
        "type": data["serviceJourney"]["line"]["transportMode"]
    }
    return stripped_item


# Create dict of format:
# (bus_line, bus_dest, platform): [departure times details]
grouped_data = defaultdict(list)
Index = namedtuple('Index', ['line', 'dst', 'platform'])
for item in data:
    line = item['serviceJourney']['line']['publicCode']
    line = int(line) if line.isdigit() else line
    quay_public_code = item['quay']['publicCode']
    line_dst = item['destinationDisplay']['frontText']
    index = Index(line, line_dst, quay_public_code)
    grouped_data[index].append(create_stripped_item(item))

grouped_data = dict(grouped_data)

# print("---")
# rnd(grouped_data)

sorted_items = sorted(grouped_data.items(), key=sort_key)  # returns list of tuples

# print("---")
# print(type(sorted_items[0]))

print("Full list.  : ", len(sorted_items))
exclusion_platform_list = ['A', 'B']
cleaned_sorted_items = [item for item in sorted_items if item[0][2] not in exclusion_platform_list]
print("Cleaned list: ", len(cleaned_sorted_items))

print_dep_times(sorted_items)

Full list.  :  25
Cleaned list:  21
Line: 5, To: Ringen via Storo, Platform: 1, Type: metro
	10:27.00 - 10:29.29
	10:42.00 - 10:42.00
Line: 5, To: Vestli, Platform: 1, Type: metro
	10:23.00 - 10:23.04
	10:38.00 - 10:38.07
Line: 5, To: Ringen via Tøyen, Platform: 2, Type: metro
	10:33.00 - 10:34.25
	10:48.00 - 10:48.00
Line: 5, To: Sognsvann via Tøyen, Platform: 2, Type: metro
	10:29.00 - 10:31.01
	10:44.00 - 10:45.04
Line: 17, To: Sinsen-Grefsen st., Platform: D, Type: tram
	10:20.00 - 10:22.28
	10:30.00 - 10:31.40
	10:40.00 - 10:40.45
	10:50.00 - 10:50.00
Line: 17, To: Rikshospitalet, Platform: E, Type: tram
	10:28.00 - 10:28.00
	10:38.00 - 10:38.00
	10:48.00 - 10:48.00
Line: 20, To: Skøyen, Platform: H, Type: bus
	10:21.00 - 10:21.56
	10:29.00 - 10:29.00
	10:36.00 - 10:36.00
	10:44.00 - 10:44.00
	10:51.00 - 10:51.00
Line: 20, To: Galgeberg, Platform: J, Type: bus
	10:21.00 - 10:21.55
	10:29.00 - 10:30.08
	10:36.00 - 10:36.10
	10:44.00 - 10:44.44
	10:51.00 - 10:51.00
Line: 21, To: Hel

In [10]:
# pretty_json = json.dumps(cleaned_sorted_items, indent=2)
# print(pretty_json)

In [13]:
out = defaultdict(dict)

for item in cleaned_sorted_items:
    # item[0] Index Tuple as key
    # item[1] Actual item/sparse data for departures
    out[item[0].line][f"{item[0].dst} - {item[0].platform}"] = item[1]


print(json.dumps(out, indent=2))

{
  "5": {
    "Ringen via Storo - 1": [
      {
        "schedule": "10:27.00",
        "expected": "10:29.29",
        "type": "metro"
      },
      {
        "schedule": "10:42.00",
        "expected": "10:42.00",
        "type": "metro"
      }
    ],
    "Vestli - 1": [
      {
        "schedule": "10:23.00",
        "expected": "10:23.04",
        "type": "metro"
      },
      {
        "schedule": "10:38.00",
        "expected": "10:38.07",
        "type": "metro"
      }
    ],
    "Ringen via T\u00f8yen - 2": [
      {
        "schedule": "10:33.00",
        "expected": "10:34.25",
        "type": "metro"
      },
      {
        "schedule": "10:48.00",
        "expected": "10:48.00",
        "type": "metro"
      }
    ],
    "Sognsvann via T\u00f8yen - 2": [
      {
        "schedule": "10:29.00",
        "expected": "10:31.01",
        "type": "metro"
      },
      {
        "schedule": "10:44.00",
        "expected": "10:45.04",
        "type": "metro"
      }
    ]
  }

In [9]:
test = {
    'foo': 'bar',
    'sna': 'foo'
}

request_args = {
    'secret': 'public'
}

if 'secret' not in request_args or request_args['secret'] != 'public':
    print("fail")